In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/DPT"
!pwd

/content/drive/MyDrive/DPT
/content/drive/MyDrive/DPT


In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# pip install -r requirements.txt
!python run_monodepth.py

initialize
device: cpu
/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
start processing
  processing input/ILSVRC2012_test_00000038.jpg (1/13)
  processing input/ILSVRC2012_test_00000184.jpg (2/13)
  processing input/ILSVRC2012_test_00000284.jpg (3/13)
  processing input/ILSVRC2012_test_00000307.jpg (4/13)
  processing input/ILSVRC2012_test_00000138.jpg (5/13)
  processing input/ILSVRC2012_test_00000378.jpg (6/13)
  processing input/ILSVRC2012_test_00061613.jpg (7/13)
  processing input/ILSVRC2012_test_00061728.jpg (8/13)
  processing input/ILSVRC2012_test_00061763.jpg (9/13)
  processing input/ILSVRC2012_test_00061700.jpg (10/13)
  processing input/ILSVRC2012_test_00063212.jpg (11/13)
  processing input/ILSVRC2012_test_00062751.jpg (12/13)
  processing input/00.jpg (13/13)
finished


In [ ]:
#Import packages
import cv2
import numpy as np
from sklearn.cluster import KMeans


# Load depth map generated from "nielsr/dpt-depth-estimation" and original RGB image
depth_map_gray = cv2.imread('/content/sample_data/img_1_depth.png', cv2.IMREAD_UNCHANGED)
rgb_image = cv2.imread('/content/sample_data/img_1_paint.jpg')

# Set depth threshold for feature segmentation
depth_threshold = 500 # please change this value with your own preference

# Generate binary mask based on depth threshold
binary_mask = np.where(depth_map_gray > depth_threshold, 255, 0).astype(np.uint8)

# Convert RGB image to grayscale for feature clustering
gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)

# Reshape the image to a 2D array of pixels
pixels = gray_image.reshape((-1, 1))

# Perform K-means clustering on the pixel values
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(pixels)

# Get the labels and cluster centers
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Reshape the labels to match the image shape
labels = labels.reshape(gray_image.shape)

# Extract cluster features from the RGB image
cluster_features = []
for i in range(num_clusters):
    mask = np.where(labels == i, 255, 0).astype(np.uint8)
    cluster_image = cv2.bitwise_and(rgb_image, rgb_image, mask=mask)
    cluster_features.append(cluster_image)

# Display the segmented image and cluster features
from matplotlib import pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(2, num_clusters + 1, 1)
plt.imshow(rgb_image[:, :, ::-1])
plt.title('Original image')
plt.axis('off')

for i in range(num_clusters):
    plt.subplot(2, num_clusters + 1, i + num_clusters + 2)
    plt.imshow(cluster_features[i][:, :, ::-1])
    plt.title(f'Cluster {i+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# Export cluster features to JSON
import json
output = {
    'num_clusters': num_clusters,
    'cluster_features': []
}

for i in range(num_clusters):
    feature_dict = {
        'cluster_id': i+1,
        'feature_data': cluster_features[i].tolist()
    }
    output['cluster_features'].append(feature_dict)

# Save the JSON data to a file
output_file = '/content/sample_data/cluster_features.json'
with open(output_file, 'w') as f:
    json.dump(output, f)

print('Cluster features exported :)')

TypeError: '>' not supported between instances of 'NoneType' and 'int'